In [ ]:
# 1.1 Load and Inspect Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load data
df = pd.read_csv('FoAI_A2_data.csv')

# Initial inspection
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")

# Focus on relevant columns for RQ1
rq1_columns = ['work_year', 'experience_level', 'job_title', 
               'salary_in_usd', 'remote_ratio', 'company_size', 'company_location']
df_rq1 = df[rq1_columns].copy()

In [ ]:
# 1.2 Data Cleaning
# Remove duplicates
df_rq1 = df_rq1.drop_duplicates()

# Handle missing values
print(f"Missing values before cleaning:\n{df_rq1.isnull().sum()}")

# Remove rows with missing critical values
df_rq1 = df_rq1.dropna(subset=['salary_in_usd', 'remote_ratio', 'experience_level'])

# Remove outliers using IQR method
def remove_outliers_iqr(df, column, multiplier=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply to salary
df_rq1 = remove_outliers_iqr(df_rq1, 'salary_in_usd')

print(f"\nDataset shape after cleaning: {df_rq1.shape}")

In [ ]:
# 1.3 Feature Engineering
# Create remote work categories for clarity
df_rq1['remote_category'] = df_rq1['remote_ratio'].map({
    0: 'On-site',
    50: 'Hybrid', 
    100: 'Fully Remote'
})

# Create experience level descriptions
experience_map = {
    'EN': 'Entry Level',
    'MI': 'Mid Level',
    'SE': 'Senior Level',
    'EX': 'Executive Level'
}
df_rq1['experience_desc'] = df_rq1['experience_level'].map(experience_map)

# Create salary bands for visualization
df_rq1['salary_band'] = pd.cut(df_rq1['salary_in_usd'], 
                                bins=[0, 75000, 125000, 175000, 250000, np.inf],
                                labels=['<75K', '75-125K', '125-175K', '175-250K', '>250K'])

# Filter to most recent years (2022-2024) for relevance
df_rq1_recent = df_rq1[df_rq1['work_year'] >= 2022].copy()

print(f"Focused dataset (2022-2024): {df_rq1_recent.shape}")

In [ ]:
# 2.1 Overall Remote Work Distribution
# Count by remote ratio
remote_dist = df_rq1_recent['remote_category'].value_counts()
print("Remote work distribution:")
print(remote_dist)
print(f"\nPercentages:")
print(remote_dist / len(df_rq1_recent) * 100)

In [ ]:
# Chart 1: Distribution of Remote Work Policies
plt.figure(figsize=(10, 6))
colors = ['#ef4444', '#f59e0b', '#10b981']
remote_dist.plot(kind='bar', color=colors)
plt.title('Distribution of Remote Work Policies (2022-2024)', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Remote Work Policy', fontsize=12)
plt.ylabel('Number of Positions', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(remote_dist):
    plt.text(i, v + 50, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('chart1_remote_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 2.2 Salary Statistics by Remote Ratio
# Calculate comprehensive statistics
salary_stats = df_rq1_recent.groupby('remote_category')['salary_in_usd'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std'),
    ('min', 'min'),
    ('25%', lambda x: x.quantile(0.25)),
    ('75%', lambda x: x.quantile(0.75)),
    ('max', 'max')
]).round(0)

print("\n" + "="*80)
print("SALARY STATISTICS BY REMOTE WORK POLICY")
print("="*80)
print(salary_stats)

# Calculate percentage differences from on-site baseline
onsite_mean = salary_stats.loc['On-site', 'mean']
salary_stats['% diff from on-site'] = ((salary_stats['mean'] - onsite_mean) / onsite_mean * 100).round(2)

print("\nPercentage difference from on-site baseline:")
print(salary_stats[['mean', '% diff from on-site']])

In [ ]:
# Chat 2: Box Plot - Salary Distribution by Remote Policy
plt.figure(figsize=(12, 7))
box_plot = sns.boxplot(data=df_rq1_recent, x='remote_category', y='salary_in_usd',
                        order=['On-site', 'Hybrid', 'Fully Remote'],
                        palette=['#ef4444', '#f59e0b', '#10b981'])

plt.title('Salary Distribution by Remote Work Policy', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Remote Work Policy', fontsize=12)
plt.ylabel('Annual Salary (USD)', fontsize=12)
plt.grid(axis='y', alpha=0.3)

# Add mean markers
means = df_rq1_recent.groupby('remote_category')['salary_in_usd'].mean()
positions = [0, 1, 2]
plt.scatter(positions, means.values, color='red', s=200, zorder=3, marker='D', 
            label='Mean Salary', edgecolors='darkred', linewidths=2)

plt.legend()
plt.tight_layout()
plt.savefig('chart2_salary_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Split data by remote category
onsite = df_rq1_recent[df_rq1_recent['remote_ratio'] == 0]['salary_in_usd']
hybrid = df_rq1_recent[df_rq1_recent['remote_ratio'] == 50]['salary_in_usd']
remote = df_rq1_recent[df_rq1_recent['remote_ratio'] == 100]['salary_in_usd']

# Perform ANOVA test
f_stat, p_value = stats.f_oneway(onsite, hybrid, remote)

print("\n" + "="*80)
print("STATISTICAL SIGNIFICANCE TEST (ANOVA)")
print("="*80)
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.6f}")
print(f"Result: {'Statistically significant' if p_value < 0.05 else 'Not significant'} at α=0.05")

# Post-hoc pairwise comparisons (Tukey HSD)
from scipy.stats import tukey_hsd
res = tukey_hsd(onsite, hybrid, remote)
print("\n" + "="*80)
print("POST-HOC PAIRWISE COMPARISONS (Tukey HSD)")
print("="*80)
print(f"Comparison results:\n{res}")

In [ ]:
# 2.3 Statistical Significance Testing
# Create pivot table
pivot_salary = df_rq1_recent.pivot_table(
    values='salary_in_usd',
    index='experience_level',
    columns='remote_category',
    aggfunc='mean'
).round(0)

# Reorder columns for better readability
pivot_salary = pivot_salary[['On-site', 'Hybrid', 'Fully Remote']]

print("\n" + "="*80)
print("AVERAGE SALARY BY EXPERIENCE LEVEL AND REMOTE POLICY")
print("="*80)
print(pivot_salary)

# Calculate percentage savings for each experience level
pivot_savings = pivot_salary.copy()
for col in ['Hybrid', 'Fully Remote']:
    pivot_savings[f'{col} Savings %'] = (
        (pivot_salary['On-site'] - pivot_salary[col]) / pivot_salary['On-site'] * 100
    ).round(2)

print("\n" + "="*80)
print("COST SAVINGS PERCENTAGE BY REMOTE POLICY")
print("="*80)
print(pivot_savings[['On-site', 'Hybrid', 'Hybrid Savings %', 
                      'Fully Remote', 'Fully Remote Savings %']])